# Incomplete Cookbook

This notebook runs IFbench benchmark on Prompt Learning. It is incomplete for now.

In [3]:
from openai import AsyncOpenAI

from phoenix.client import Client
from phoenix.client.experiments import async_run_experiment

from phoenix.evals import create_evaluator
from phoenix.evals.llm import LLM

import os
import pandas as pd
import requests
import json
from pathlib import Path
import sys

/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
if_test_df = pd.read_parquet("hf://datasets/allenai/IFBench_test/data/train-00000-of-00001.parquet").sample(294, random_state=42)
if_train = pd.read_parquet("hf://datasets/allenai/IF_multi_constraints_upto5/data/train-00000-of-00001.parquet")
if_train_df = if_train.sample(150, random_state=42)
if_val_df = if_train.drop(if_train_df.index).sample(300, random_state=42)

In [1]:
create_prompt = """
Respond to the query.

Query: {query}
"""

follow_instructions_prompt = """
Ensure the response is correct and adheres to the given constraints. Your
response will be used as the final response.

Response: {response}
Constraints: {constraint}
"""

In [ ]:
phoenix_client = Client()

# phoenix_train_dataset = phoenix_client.datasets.create_dataset(
#     name="IFBENCH_TRAIN",
#     dataset_description="IFBench Training Set",
#     dataframe=if_train_df,
#     input_keys=["messages", "constraint"],
#     output_keys=["ground_truth"],
#     metadata_keys=["key", "dataset", "constraint_type"]
# )

# phoenix_validation_dataset = phoenix_client.datasets.create_dataset(
#     name="IFBENCH_VALIDATION",
#     dataset_description="IFBench Validation Set",
#     dataframe=if_val_df,
#     input_keys=["messages", "constraint"],
#     output_keys=["ground_truth"],
#     metadata_keys=["key", "dataset", "constraint_type"]
# )

phoenix_train_dataset = phoenix_client.datasets.get_dataset(dataset="IFBENCH_TRAIN")
phoenix_validation_dataset = phoenix_client.datasets.get_dataset(dataset="IFBENCH_VALIDATION")

# phoenix_test_dataset = phoenix_client.datasets.create_dataset(
#     name="IFBENCH_TEST",
#     dataset_description="IFBench Test Set",
#     dataframe=if_test_df,
#     input_keys=["prompt", "instruction_id_list"],
#     output_keys=["kwargs"],
#     metadata_keys=["key"]
# )


In [5]:
def task_maker(create_prompt, follow_instructions_prompt):
    async def task(input):
        # try:
        client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        formatted_create_prompt = create_prompt.format(query=input["messages"])
        response = await client.responses.create(
            model="gpt-4.1-mini",
            input=formatted_create_prompt
        )
        response = response.output_text
        formatted_follow_instructions_prompt = follow_instructions_prompt.format(response=response, constraint=input["constraint"])
        final_response = await client.responses.create(
            model="gpt-4.1-mini",
            input=formatted_follow_instructions_prompt
        )
        final_response = final_response.output_text
        # print("working")
        return {
            "response": response,
            "final_response": final_response
            }
        # except Exception as e:
        #     print(e)
        #     return {
        #         "response": "",
        #         "final_response": ""
        #     }
    return task




In [ ]:
from openai import RateLimitError
from phoenix.client.experiments import async_run_experiment
from pathlib import Path
import sys
from evals import evaluate_response
project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

# EXPERIMENT HAS ASYNCIO.CANCELLEDERROR

train_experiment = await async_run_experiment(
    task=task_maker(create_prompt, follow_instructions_prompt),
    dataset=phoenix_train_dataset,
    evaluators=[evaluate_response],
    concurrency=10,
    rate_limit_errors=RateLimitError,
)

In [8]:
import requests
import pprint
import json
import pandas as pd

def extract_judge_values(annots):
    if not isinstance(annots, list):
        return pd.Series({"number_of_pii_leaked": None, "response_quality": None})
    
    compliance = None
    compliance_explanation = None
    
    for a in annots:
        name = a.get("name")
        if name == "evaluator":
            compliance = a.get("label")
            compliance_explanation = a.get("explanation")
    return pd.Series({
        "compliance": compliance,
        "compliance_explanation": compliance_explanation
    })

def process_experiment(experiment, dataset_name):
    experiment_id = experiment["experiment_id"]
    url = f"http://localhost:6006/v1/experiments/{experiment_id}/json"

    response = requests.get(url)
    experiment_results = response.json()
    experiment_results_json = json.dumps(experiment_results)
    experiment_results_df = pd.read_json(experiment_results_json)

    dataset = phoenix_client.datasets.get_dataset(dataset=dataset_name)
    dataset_df = dataset.to_dataframe()
    dataset_df.rename(columns={"output": "expected"}, inplace=True)
    subset_df = dataset_df[["expected", "metadata"]]
    experiment_results = experiment_results_df.merge(subset_df, left_on="example_id", right_index=True, how="left")

    df = experiment_results.copy()
# --- remove rows with missing structured fields ---
    df = df.dropna(subset=["input", "output", "metadata", "expected"])
    
    # Create judge_df from the filtered df to ensure indices match
    judge_df = df["annotations"].apply(extract_judge_values)
    
    dataset = pd.DataFrame({
        "query": df["input"].apply(lambda x: x.get("messages")),
        "constraint": df["input"].apply(lambda x: x.get("constraint")),
        "messages": df["input"].apply(lambda x: x.get("messages")),
        "response": df["output"].apply(lambda x: x.get("response")),
        "final_response": df["output"].apply(lambda x: x.get("final_response")),
    })

    final_experiment_results = pd.concat([dataset, judge_df], axis=1)
    return final_experiment_results
    
    


In [9]:
META_PROMPT_IFBENCH = """
You are optimizing prompts for a **constraint following LLM**.

====================
LLM SYSTEM OVERVIEW
====================
Step 1: Answer the query, using the first prompt.
Step 2: Ensure the response adheres to the given constraints, using the second prompt.

Example data includes:
- constraint → constraint that the response must adhere to
- messages → user query
- response → response from the the first prompt
- final_response → final response, generated by the second prompt

====================
OPTIMIZATION GOAL
====================
Improve the prompts guiding the LLM so that:
- Responses become more accurate and adhere to the given constraints.

====================
INPUTS
====================
************* start prompts *************
{baseline_prompt}
************* end prompt *************

************* start example data *************
{examples}
************* end example data *************

- Keep placeholders (variables in single brackets) exactly as written. If you remove these, the pipeline will break. Don't add anything else in single brackets. 
THERE SHOULD BE NO BRACKETS IN THE PROMPT YOU PRODUCE, EITHER THAN THOSE SURROUNDING THE VARIABLES.
- Maintain the original return format and structure.  
- Add clear reasoning or few shot examples if helpful.

New prompts:
"""


In [ ]:
from openai import RateLimitError
from phoenix.client.experiments import async_run_experiment
from pathlib import Path
import sys
from evals import evaluate_response
project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

# prompts = []
# prompts.append({"create_prompt": create_prompt, "follow_instructions_prompt": follow_instructions_prompt})

for i in range(1):

    train_experiment = await async_run_experiment(
        task=task_maker(create_prompt, follow_instructions_prompt),
        dataset=phoenix_train_dataset,
        evaluators=[evaluate_response],
        concurrency=10,
        rate_limit_errors=RateLimitError,
    )

    # validation_experiment = await async_run_experiment(
    #     task=task_maker(create_prompt, follow_instructions_prompt),
    #     dataset=phoenix_validation_dataset,
    #     evaluators=[evaluate_response],
    #     concurrency=5
    # )

    # train_df = process_experiment(train_experiment, "IFBENCH_TRAIN")
    # prompts_concatenated = "create_prompt: " + create_prompt + "\nfollow_instructions_prompt: " + follow_instructions_prompt
    # pl_optimizer = PromptLearningOptimizer(
    #     prompt=prompts_concatenated,
    #     model_choice="gpt-5",
    #     meta_prompt = META_PROMPT_IFBENCH
    # )

    # optimized_prompts = pl_optimizer.optimize(
    #     dataset=train_df,
    #     output_column="final_response",
    #     feedback_columns=["compliance", "compliance_explanation"],
    #     context_size_k=300000
    # )

    # text = optimized_prompts.strip("'")
    # parts = text.split("follow_instructions_prompt:", 1)
    # create_prompt = parts[0].replace("create_prompt:", "").strip()
    # follow_instructions_prompt = parts[1].strip() if len(parts) > 1 else None
    # prompts.append({"create_prompt": create_prompt, "follow_instructions_prompt": follow_instructions_prompt})



🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDoxNg==/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoxNg==/compare?experimentId=RXhwZXJpbWVudDoxMDY=


running tasks |██        | 31/150 (20.7%) | ⏳ 01:00<02:46 |  1.40s/it 

Worker timeout, requeuing
Traceback (most recent call last):
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/__init__.py", line 3414, in _run_single_task_async
    output = await _output
  File "/var/folders/_w/glgvmwgs3s5g81607b0x435c0000gn/T/ipykernel_20884/546940153.py", line 6, in task
    response = await client.responses.create(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/resources/responses/responses.py", line 2259, in create
    return await self._post(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/_base_client.py", line 1794, in post
    return await self.request(cast_to, opts, stream=stream, stream_cls=stream_cls)
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/_base_client.py", line 1529, in request
    response = await self._client.send(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/httpx/_client.py", line 1629, in

running tasks |████      | 61/150 (40.7%) | ⏳ 02:00<02:06 |  1.42s/it 

Worker timeout, requeuing
Traceback (most recent call last):
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/__init__.py", line 3414, in _run_single_task_async
    output = await _output
  File "/var/folders/_w/glgvmwgs3s5g81607b0x435c0000gn/T/ipykernel_20884/546940153.py", line 6, in task
    response = await client.responses.create(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/resources/responses/responses.py", line 2259, in create
    return await self._post(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/_base_client.py", line 1794, in post
    return await self.request(cast_to, opts, stream=stream, stream_cls=stream_cls)
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/openai/_base_client.py", line 1529, in request
    response = await self._client.send(
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/site-packages/httpx/_client.py", line 1629, in

...

running experiment evaluations |▍         | 5/124 (4.0%) | ⏳ 00:04<01:57 |  1.02it/s

Traceback (most recent call last):
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/__init__.py", line 3580, in _run_single_evaluation_async
    result = await evaluator.async_evaluate(
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/evaluators.py", line 260, in async_evaluate
    scores = await evaluator.async_evaluate(kwargs)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 269, in async_evaluate
    return await self._async_evaluate(remapped_eval_input)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 218, in _async_evaluate
    result = await to_thread(self._evaluate)(eval_input)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 147, in wrapper
    return await loop.run_in_executor(None, lambda: fn(*args, **kwargs))
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result

running experiment evaluations |▋         | 8/124 (6.5%) | ⏳ 00:10<02:41 |  1.39s/it

Traceback (most recent call last):
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/__init__.py", line 3580, in _run_single_evaluation_async
    result = await evaluator.async_evaluate(
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/client/resources/experiments/evaluators.py", line 260, in async_evaluate
    scores = await evaluator.async_evaluate(kwargs)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 269, in async_evaluate
    return await self._async_evaluate(remapped_eval_input)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 218, in _async_evaluate
    result = await to_thread(self._evaluate)(eval_input)
  File "/Users/priyanjindal/phoenix-oss/src/phoenix/evals/evaluators.py", line 147, in wrapper
    return await loop.run_in_executor(None, lambda: fn(*args, **kwargs))
  File "/opt/anaconda3/envs/cancelled-error/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result

running experiment evaluations |▋         | 9/124 (7.3%) | ⏳ 00:10<02:03 |  1.07s/it

In [ ]:
create_prompt = 'You are an expert assistant. Your job in this step is to produce the best possible core answer to the user’s request—focusing on accuracy, completeness, and clarity—while not worrying about strict formatting or token-level constraints. A later step will enforce all special formatting and constraint rules.\n\nDo the following, silently:\n1) Parse the question carefully. Identify:\n   - All explicit tasks and sub-questions.\n   - Data, variables, assumptions, and definitions (units, domains).\n   - Target outputs and what must be computed, proved, derived, or designed.\n   - Any safety, legality, or ethics issues that would require refusal or a safe alternative.\n2) Plan a minimal, correct solution for each sub-task:\n   - Show key reasoning, steps, and formulas only where it improves correctness or transparency.\n   - For math: use standard symbols, correct units, and simplified final forms.\n   - For code: provide a minimal, correct, idiomatic solution; include brief comments for clarity.\n   - For design or writing: provide clear structure and directly address requirements.\n3) Compute results accurately:\n   - Double-check calculations, dimensions, and edge cases.\n   - Where possible, verify by an alternate method or a quick consistency check.\n4) Prepare content that is easy to transform later:\n   - Prefer short, self-contained sentences or bullet points with the needed facts.\n   - Keep terminology consistent so downstream formatting is reliable.\n   - If the final answer will need specific counts (e.g., bullets, sentences), provide all necessary facts but do not enforce the counts here.\n5) If the request is disallowed or unsafe:\n   - Provide a brief, clear refusal with a safe, constructive alternative aligned to the user’s apparent intent (e.g., “I can’t help with X, but I can help with Y.”).\n6) If a specific language or style is requested, produce the core content in that language or style to the extent possible, but do not enforce mechanical constraints (case, punctuation bans, last-word rules, hyphen-only, bigram wrappers, JSON shells, exact counts, etc.) in this step.\n7) Do not attempt to satisfy strict output constraints (counts, wrappers, casing, banned punctuation/words, last-word rules, “exactly N bullets,” JSON keys, etc.) in this step. That will be handled later.\n\nWhen helpful, structure your core answer with clear sections, short paragraphs, or bullet points so the next step can transform it to meet exact constraints.'
follow_instructions_prompt = 'You are finalizing the response. Your task is to transform or regenerate the content so that it is both correct and FULLY compliant with ALL given constraints. Use the “Response” content as raw material when helpful, but you may rewrite from scratch if necessary. Output only the final compliant text.\n\nPerform these steps silently and meticulously:\n\nSTEP 1 — Extract and Formalize ALL Constraints\n- Read “Constraints” and “Messages.” List every requirement precisely, including:\n  - Structure: exact counts of sections, bullets, sentences, paragraphs, lines; required separators; wrappers or code fences; titles; required prefixes/suffixes; start/end tokens; required first/last words.\n  - Lexical/typographic rules: required or banned keywords; language; case rules; punctuation bans; required symbols; forbidden characters; numeric or letter frequency limits; uniqueness limits; placeholder tokens; bigram or hyphen rules.\n  - Content rules: required computations/results; correctness of math and logic; inclusion or exclusion of examples; safety constraints; refusal requirements.\n- Convert these into a checklist with concrete, countable targets (e.g., “exactly 3 bullets,” “ALL CAPS,” “no commas,” “include keyword Y twice,” “last word must be X,” “every word bigram wrapped as <<w_i w_i+1>>,” “paragraph 2 starts with Z,” etc.).\n- Note dependencies and interactions (e.g., language affects casing; punctuation bans affect sentence delimiting).\n\nSTEP 2 — Plan a Compliant Skeleton Before Writing\n- Build a structure plan matching all numeric constraints: number of paragraphs/sections/sentences/bullets/lines and required separators.\n- Pre-allocate mandatory tokens and positions:\n  - First/last words per sentence or document.\n  - Exact keyword counts and their positions.\n  - Required wrappers (JSON, quotes, code fences, titles).\n  - Placeholders that must appear verbatim in single brackets.\n- If uniqueness or frequency caps exist, design a small controlled vocabulary and track usage to prevent violations.\n- If punctuation or character bans exist, plan phrasing that avoids them and choose allowed delimiters (e.g., hyphens) to count sentences.\n\nSTEP 3 — Generate Correct Content Under All Constraints\n- If results are required, compute them accurately (reuse “Response” calculations if correct; otherwise recompute succinctly).\n- Prioritize hard mechanical constraints (counts, bans, exact words) while preserving correctness.\n- Apply special mechanics precisely:\n  - Hyphen-only sentences with no spaces if required.\n  - Bigram wrapping: produce tokens as <<w_i w_i+1>> for each consecutive two-word sequence across the text, observing spacing and punctuation bans.\n  - Word uniqueness: ensure no word repeats if required; track in a set.\n  - First/last word rules per sentence and for the whole output.\n  - Letter/word frequency limits: adjust synonyms or restructure to fit limits.\n  - Exact bullets formatting (e.g., “* ”) and exact counts.\n  - Required language and casing (ALL CAPS, all lowercase, specific language).\n  - Placeholders: include required placeholders [LIKE THIS] exactly; do not invent or alter single-bracket tokens.\n- If a constraint makes full correctness impossible, produce the closest compliant text that conveys the minimally correct required result without breaking any rule.\n\nSTEP 4 — Safety and Policy Compliance\n- If the content requested is unsafe/disallowed, produce a brief, compliant refusal that still obeys ALL structure/format constraints (e.g., exact bullets, casing, wrappers) and, if possible, provide a safe alternative within the same format.\n\nSTEP 5 — Rigorous Verification Pass\n- Re-check each checklist item mechanically:\n  - Counts of bullets/sections/sentences/paragraphs/lines and required separators.\n  - Required start/end tokens; required first/last words for each sentence/document.\n  - Language and casing exactly as specified.\n  - Punctuation or character bans obeyed.\n  - Exact keyword inclusion counts and positions.\n  - Forbidden words/characters absent.\n  - Word uniqueness or letter-frequency limits satisfied.\n  - Special formatting (bigrams, hyphens, wrappers, JSON keys) applied globally and correctly.\n  - Required placeholders preserved exactly.\n- If any item fails, revise and re-verify until fully compliant.\n\nImportant:\n- Output ONLY the final compliant response text. Do not include reasoning, checklists, or notes.\n- Preserve single-bracket placeholders exactly as given when required.\n- Be concise where allowed; favor the shortest text that fully satisfies all constraints.\n- If the original “Response” is unusable, regenerate from scratch using “Messages” and “Constraints.”'

In [ ]:
prompts[-1] = {
    'create_prompt' : 'You are an expert assistant. Your job in this step is to produce the best possible core answer to the user’s request—focusing on accuracy, completeness, and clarity—while not worrying about strict formatting or token-level constraints. A later step will enforce all special formatting and constraint rules.\n\nDo the following, silently:\n1) Parse the question carefully. Identify:\n   - All explicit tasks and sub-questions.\n   - Data, variables, assumptions, and definitions (units, domains).\n   - Target outputs and what must be computed, proved, derived, or designed.\n   - Any safety, legality, or ethics issues that would require refusal or a safe alternative.\n2) Plan a minimal, correct solution for each sub-task:\n   - Show key reasoning, steps, and formulas only where it improves correctness or transparency.\n   - For math: use standard symbols, correct units, and simplified final forms.\n   - For code: provide a minimal, correct, idiomatic solution; include brief comments for clarity.\n   - For design or writing: provide clear structure and directly address requirements.\n3) Compute results accurately:\n   - Double-check calculations, dimensions, and edge cases.\n   - Where possible, verify by an alternate method or a quick consistency check.\n4) Prepare content that is easy to transform later:\n   - Prefer short, self-contained sentences or bullet points with the needed facts.\n   - Keep terminology consistent so downstream formatting is reliable.\n   - If the final answer will need specific counts (e.g., bullets, sentences), provide all necessary facts but do not enforce the counts here.\n5) If the request is disallowed or unsafe:\n   - Provide a brief, clear refusal with a safe, constructive alternative aligned to the user’s apparent intent (e.g., “I can’t help with X, but I can help with Y.”).\n6) If a specific language or style is requested, produce the core content in that language or style to the extent possible, but do not enforce mechanical constraints (case, punctuation bans, last-word rules, hyphen-only, bigram wrappers, JSON shells, exact counts, etc.) in this step.\n7) Do not attempt to satisfy strict output constraints (counts, wrappers, casing, banned punctuation/words, last-word rules, “exactly N bullets,” JSON keys, etc.) in this step. That will be handled later.\n\nWhen helpful, structure your core answer with clear sections, short paragraphs, or bullet points so the next step can transform it to meet exact constraints.',
'follow_instructions_prompt' : 'You are finalizing the response. Your task is to transform or regenerate the content so that it is both correct and FULLY compliant with ALL given constraints. Use the “Response” content as raw material when helpful, but you may rewrite from scratch if necessary. Output only the final compliant text.\n\nPerform these steps silently and meticulously:\n\nSTEP 1 — Extract and Formalize ALL Constraints\n- Read “Constraints” and “Messages.” List every requirement precisely, including:\n  - Structure: exact counts of sections, bullets, sentences, paragraphs, lines; required separators; wrappers or code fences; titles; required prefixes/suffixes; start/end tokens; required first/last words.\n  - Lexical/typographic rules: required or banned keywords; language; case rules; punctuation bans; required symbols; forbidden characters; numeric or letter frequency limits; uniqueness limits; placeholder tokens; bigram or hyphen rules.\n  - Content rules: required computations/results; correctness of math and logic; inclusion or exclusion of examples; safety constraints; refusal requirements.\n- Convert these into a checklist with concrete, countable targets (e.g., “exactly 3 bullets,” “ALL CAPS,” “no commas,” “include keyword Y twice,” “last word must be X,” “every word bigram wrapped as <<w_i w_i+1>>,” “paragraph 2 starts with Z,” etc.).\n- Note dependencies and interactions (e.g., language affects casing; punctuation bans affect sentence delimiting).\n\nSTEP 2 — Plan a Compliant Skeleton Before Writing\n- Build a structure plan matching all numeric constraints: number of paragraphs/sections/sentences/bullets/lines and required separators.\n- Pre-allocate mandatory tokens and positions:\n  - First/last words per sentence or document.\n  - Exact keyword counts and their positions.\n  - Required wrappers (JSON, quotes, code fences, titles).\n  - Placeholders that must appear verbatim in single brackets.\n- If uniqueness or frequency caps exist, design a small controlled vocabulary and track usage to prevent violations.\n- If punctuation or character bans exist, plan phrasing that avoids them and choose allowed delimiters (e.g., hyphens) to count sentences.\n\nSTEP 3 — Generate Correct Content Under All Constraints\n- If results are required, compute them accurately (reuse “Response” calculations if correct; otherwise recompute succinctly).\n- Prioritize hard mechanical constraints (counts, bans, exact words) while preserving correctness.\n- Apply special mechanics precisely:\n  - Hyphen-only sentences with no spaces if required.\n  - Bigram wrapping: produce tokens as <<w_i w_i+1>> for each consecutive two-word sequence across the text, observing spacing and punctuation bans.\n  - Word uniqueness: ensure no word repeats if required; track in a set.\n  - First/last word rules per sentence and for the whole output.\n  - Letter/word frequency limits: adjust synonyms or restructure to fit limits.\n  - Exact bullets formatting (e.g., “* ”) and exact counts.\n  - Required language and casing (ALL CAPS, all lowercase, specific language).\n  - Placeholders: include required placeholders [LIKE THIS] exactly; do not invent or alter single-bracket tokens.\n- If a constraint makes full correctness impossible, produce the closest compliant text that conveys the minimally correct required result without breaking any rule.\n\nSTEP 4 — Safety and Policy Compliance\n- If the content requested is unsafe/disallowed, produce a brief, compliant refusal that still obeys ALL structure/format constraints (e.g., exact bullets, casing, wrappers) and, if possible, provide a safe alternative within the same format.\n\nSTEP 5 — Rigorous Verification Pass\n- Re-check each checklist item mechanically:\n  - Counts of bullets/sections/sentences/paragraphs/lines and required separators.\n  - Required start/end tokens; required first/last words for each sentence/document.\n  - Language and casing exactly as specified.\n  - Punctuation or character bans obeyed.\n  - Exact keyword inclusion counts and positions.\n  - Forbidden words/characters absent.\n  - Word uniqueness or letter-frequency limits satisfied.\n  - Special formatting (bigrams, hyphens, wrappers, JSON keys) applied globally and correctly.\n  - Required placeholders preserved exactly.\n- If any item fails, revise and re-verify until fully compliant.\n\nImportant:\n- Output ONLY the final compliant response text. Do not include reasoning, checklists, or notes.\n- Preserve single-bracket placeholders exactly as given when required.\n- Be concise where allowed; favor the shortest text that fully satisfies all constraints.\n- If the original “Response” is unusable, regenerate from scratch using “Messages” and “Constraints.”'
}

In [ ]:
prompts